In [101]:
import pandas as pd
df = pd.read_csv("Food_Time new.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Traffic_Level                9085 non-null   object 
 1   ID                           10000 non-null  object 
 2   Delivery_person_ID           10000 non-null  object 
 3   weather_description          9995 non-null   object 
 4   Type_of_order                10000 non-null  object 
 5   Type_of_vehicle              10000 non-null  object 
 6   Delivery_person_Age          10000 non-null  int64  
 7   Delivery_person_Ratings      10000 non-null  float64
 8   Restaurant_latitude          10000 non-null  object 
 9   Restaurant_longitude         10000 non-null  object 
 10  Delivery_location_latitude   10000 non-null  object 
 11  Delivery_location_longitude  10000 non-null  object 
 12  temperature                  9995 non-null   float64
 13  humidity         

In [102]:
df.describe()

,Delivery_person_Age,Delivery_person_Ratings,temperature,humidity,precipitation,Distance (km)
count,10000.000000,10000.000000,9995.000000,9995.000000,9995.000000,9080.000000
mean,29.522000,4.629370,22.936907,66.164882,0.016233,14.292747
std,5.700348,0.322941,3.379448,15.602939,0.074911,8.376254
min,15.000000,1.000000,6.770000,27.000000,0.000000,1.550000
25%,25.000000,4.500000,20.140000,54.000000,0.000000,7.620000
50%,29.000000,4.700000,23.000000,67.000000,0.000000,13.400000
75%,34.000000,4.800000,25.980000,81.000000,0.000000,19.610000
max,50.000000,6.000000,29.050000,94.000000,1.460000,59.840000


In [103]:
df.nunique()

Traffic_Level                     5
ID                             9995
Delivery_person_ID             1285
weather_description              11
Type_of_order                     4
Type_of_vehicle                   4
Delivery_person_Age              22
Delivery_person_Ratings          28
Restaurant_latitude             485
Restaurant_longitude            427
Delivery_location_latitude     3445
Delivery_location_longitude    3445
temperature                    1069
humidity                         64
precipitation                    35
Distance (km)                  2343
TARGET                         3370
dtype: int64

In [104]:
df.isnull().sum()

Traffic_Level                  915
ID                               0
Delivery_person_ID               0
weather_description              5
Type_of_order                    0
Type_of_vehicle                  0
Delivery_person_Age              0
Delivery_person_Ratings          0
Restaurant_latitude              0
Restaurant_longitude             0
Delivery_location_latitude       0
Delivery_location_longitude      0
temperature                      5
humidity                         5
precipitation                    5
Distance (km)                  920
TARGET                         541
dtype: int64

In [105]:
df = df.dropna(subset=["weather_description"])

In [106]:
df.isnull().sum()

Traffic_Level                  915
ID                               0
Delivery_person_ID               0
weather_description              0
Type_of_order                    0
Type_of_vehicle                  0
Delivery_person_Age              0
Delivery_person_Ratings          0
Restaurant_latitude              0
Restaurant_longitude             0
Delivery_location_latitude       0
Delivery_location_longitude      0
temperature                      0
humidity                         0
precipitation                    0
Distance (km)                  920
TARGET                         536
dtype: int64

In [107]:
df_median = df["Distance (km)"].median()

In [108]:
df["Distance (km)"] = df["Distance (km)"].fillna(df_median)

In [109]:
import numpy as np
df.loc[df["TARGET"].str.contains("#", na=False), "TARGET"] = np.nan
df["TARGET"] = df["TARGET"].str.replace(".", "", regex=False)
df = df.dropna(subset=["TARGET"])
df["TARGET"] = df["TARGET"].astype(int)
def fix_target(value):
    value = float(value)
    if value < 100000:
        result = value / 100
    else:
        result = value / 100000000
    return int(round(result))  # arrotonda al minuto più vicino
df["TARGET"] = df["TARGET"].apply(fix_target)

In [110]:
def fix_coord(col):
    return (
        df[col]
        .astype(str)
        .str.replace(".", "", regex=False)
        .astype(int)
        / 1_000_000
    )

coord_cols = [
    "Restaurant_latitude", "Restaurant_longitude",
    "Delivery_location_latitude", "Delivery_location_longitude"
]

for col in coord_cols:
    df[col] = fix_coord(col)

if "ID" in df.columns:
    df = df.drop(columns=["ID"])


In [111]:
df.info()
df["TARGET"].head(30)


<class 'pandas.core.frame.DataFrame'>
Index: 9040 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Traffic_Level                9040 non-null   object 
 1   Delivery_person_ID           9040 non-null   object 
 2   weather_description          9040 non-null   object 
 3   Type_of_order                9040 non-null   object 
 4   Type_of_vehicle              9040 non-null   object 
 5   Delivery_person_Age          9040 non-null   int64  
 6   Delivery_person_Ratings      9040 non-null   float64
 7   Restaurant_latitude          9040 non-null   float64
 8   Restaurant_longitude         9040 non-null   float64
 9   Delivery_location_latitude   9040 non-null   float64
 10  Delivery_location_longitude  9040 non-null   float64
 11  temperature                  9040 non-null   float64
 12  humidity                     9040 non-null   float64
 13  precipitation          

0     43
1     38
2     36
3     49
4     52
5     35
6     44
7     54
8     47
9     47
10    46
11    41
12    44
13    46
14    50
15    50
16     4
17    45
18    46
19    36
20    40
21    48
22     5
23    47
24    53
25    52
26     3
27     4
28    46
29    41
Name: TARGET, dtype: int64

In [112]:
df["Delivery_location_latitude"].info()
df["Delivery_location_longitude"].info()

<class 'pandas.core.series.Series'>
Index: 9040 entries, 0 to 9999
Series name: Delivery_location_latitude
Non-Null Count  Dtype  
--------------  -----  
9040 non-null   float64
dtypes: float64(1)
memory usage: 141.2 KB
<class 'pandas.core.series.Series'>
Index: 9040 entries, 0 to 9999
Series name: Delivery_location_longitude
Non-Null Count  Dtype  
--------------  -----  
9040 non-null   float64
dtypes: float64(1)
memory usage: 141.2 KB
